
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 2


**Student Name(s):**    `Yanze Mao, Dong Cheng Ding

**Student ID(s):**     `988142 952328

In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_validate

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn import datasets, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [9]:
# load datasets
review_text_test = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_text_test.csv")
review_text_train = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_text_train.csv")
review_text_test_meta = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_meta_test.csv")
review_text_train_meta = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_meta_train.csv")
review50=pd.read_csv("COMP30027_2020_assignment2_datasets\\review_text_train_doc2vec50.csv",header=None)
review100=pd.read_csv("COMP30027_2020_assignment2_datasets\\review_text_train_doc2vec100.csv",header=None)


## Navie Bayes

In [70]:
# MultinomialNB with countvectorizer

NB_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                ("tfidf",TfidfTransformer()),
                ("clf", MultinomialNB())])
NBparameters={
     'tfidf__use_idf':(True,False),
    'clf__alpha':[0,0.001,0.01,0.1,0.2],
    'clf__fit_prior':[True,False]
}
NB_clf = GridSearchCV(NB_clf,NBparameters, cv = 5, n_jobs=-1)
NB_clf=NB_clf.fit(review_text_train["review"], review_text_train_meta["rating"])

In [71]:
NB_clf.best_params_

{'clf__alpha': 0.1, 'clf__fit_prior': False, 'tfidf__use_idf': False}

In [72]:
NB_clf.best_score_

0.8413852073535699

In [9]:
# MultinomialNB with vect50,100,200

NB_clf=Pipeline([
                ("clf", GaussianNB())])
NBparameters={
    'clf__var_smoothing':[1e-9,1e-8]
}
NB_clf = GridSearchCV(NB_clf,NBparameters, cv = 5, n_jobs=-1)
NB_clf=NB_clf.fit(review50, review_text_train_meta["rating"])

In [10]:
NB_clf.best_params_

{'clf__var_smoothing': 1e-09}

In [11]:
NB_clf.best_score_

0.6701938150206641

## Neural Network

In [61]:
# after adjustment of attributes, the best estimator with countervect:

MLP_clf = Pipeline([('vect',CountVectorizer(stop_words="english",decode_error='ignore')),
                    ("tfidf",TfidfTransformer(use_idf=True)),
                    ('clf',MLPClassifier(activation='relu', alpha=1e-05,
       hidden_layer_sizes=(80,100,100), solver='lbfgs'))])


scores2 = cross_validate(MLP_clf, review_text_train["review"], review_text_train_meta["rating"], cv = 5)
scores2["test_score"]

array([0.84683882, 0.85518347, 0.84574279, 0.8376982 , 0.8492516 ])

In [62]:
scores2["test_score"].mean()

0.8469429769356409

In [38]:
# after adjustment of attributes, the best estimator with vect50,100,200:

MLP_clf = Pipeline([
                    ('clf',MLPClassifier(activation='relu', alpha=1e-05,
       hidden_layer_sizes=(80,100,100), solver='lbfgs'))])


scores2 = cross_validate(MLP_clf, review50, review_text_train_meta["rating"], cv = 5)
scores2["test_score"]

array([0.75494212, 0.77075169, 0.76505166, 0.76679138, 0.76799715])

In [40]:
# after adjustment of attributes, the best estimator with vect50,100,200:

MLP_clf = Pipeline([
                    ('clf',MLPClassifier(solver='lbfgs'))])


scores2 = cross_validate(MLP_clf, review100, review_text_train_meta["rating"], cv = 5)
scores2["test_score"]

array([0.76028495, 0.76237976, 0.7680798 , 0.76946374, 0.76853172])

## SVM

In [12]:
#SVM with countervect
SVM_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                ("tfidf",TfidfTransformer(use_idf=True)),
                ("clf", svm.SVC(kernel='linear', gamma=0.7, C=1.0))])

SVMparameters={
    "clf__gamma":[0.6,0.7,0.5,0.4],
    "clf__C":[0.5,0.7,1.0,2.0]
}
SVMgs_clf = GridSearchCV(SVM_clf,SVMparameters, cv = 5, n_jobs=-1)
SVMgs_clf=SVMgs_clf.fit(review_text_train["review"], review_text_train_meta["rating"])

In [15]:
SVMgs_clf.best_params_

{'clf__C': 1.0, 'clf__gamma': 0.6}

In [ ]:
SVMgs_clf.best_score_

In [5]:
#SVM with vect100
SVM_clf=Pipeline([
                ("clf", svm.SVC(kernel='linear', gamma=0.7, C=1.0))])

SVMparameters={
    "clf__gamma":[0.6,0.7,0.5,0.4],
    "clf__C":[0.5,0.7,1.0,2.0]
}
SVMgs_clf = GridSearchCV(SVM_clf,SVMparameters, cv = 5, n_jobs=-1)
SVMgs_clf=SVMgs_clf.fit(review100, review_text_train_meta["rating"])

In [6]:
SVMgs_clf.best_score_

0.8265640587145504

In [7]:
#SVM with vect100
SVM_clf=Pipeline([
                ("clf", svm.SVC(kernel='linear', gamma=0.7, C=1.0))])

SVMparameters={
    "clf__gamma":[0.6,0.7,0.5,0.4],
    "clf__C":[0.5,0.7,1.0,2.0]
}
SVMgs_clf = GridSearchCV(SVM_clf,SVMparameters, cv = 5, n_jobs=-1)
SVMgs_clf=SVMgs_clf.fit(review50, review_text_train_meta["rating"])

In [8]:
SVMgs_clf.best_score_

0.8165170300698305

In [9]:
SVMgs_clf.best_params_

{'clf__C': 0.7, 'clf__gamma': 0.6}

## RandomForest

In [19]:
# randomforestclassifier with countervect
RF_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                ("tfidf",TfidfTransformer(use_idf=True)),
                ("clf", RandomForestClassifier())])

RFparameters={
    "tfidf__use_idf":[True,False],
    "clf__n_estimators":[100,300,500],
    "clf__criterion":["gini","entropy"],
    "clf__max_depth":[10,15,20,30],
    "clf__min_samples_split":[2,4,6],
}
RF_clf = GridSearchCV(RF_clf,RFparameters, cv = 5, n_jobs=-1)
RF_clf=RF_clf.fit(review_text_train["review"], review_text_train_meta["rating"])

In [23]:
RF_clf.best_params_

{'clf__criterion': 'gini',
 'clf__max_depth': 30,
 'clf__min_samples_split': 2,
 'clf__n_estimators': 100,
 'tfidf__use_idf': False}

In [22]:
RF_clf.best_score_

0.6955251531993729

In [30]:
# randomforestclassifier with countervect
RF_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                ("tfidf",TfidfTransformer(use_idf=True)),
                ("clf", RandomForestClassifier())])

RFparameters={
    "tfidf__use_idf":[False],
    "clf__criterion":["gini","entropy"],
    "clf__min_samples_split":[2,4,6]
}
RF_clf = GridSearchCV(RF_clf,RFparameters, cv = 5, n_jobs=-1)
RF_clf=RF_clf.fit(review_text_train["review"], review_text_train_meta["rating"])
RF_clf.best_score_

H:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7509975773122417

In [31]:
RF_clf.best_params_

{'clf__criterion': 'gini',
 'clf__min_samples_split': 2,
 'tfidf__use_idf': False}

In [42]:
# randomforestclassifier with vect50
RF_clf=Pipeline([
                ("clf", RandomForestClassifier())])

RFparameters={
    "clf__criterion":["gini","entropy"],
    "clf__min_samples_split":[6,8,9,10,12]
}
RF_clf = GridSearchCV(RF_clf,RFparameters, cv = 5, n_jobs=-1)
RF_clf=RF_clf.fit(review50, review_text_train_meta["rating"])
RF_clf.best_score_

H:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7535271483539975

In [43]:
RF_clf.best_params_

{'clf__criterion': 'entropy', 'clf__min_samples_split': 9}

In [46]:
# randomforestclassifier with vect100
RF_clf=Pipeline([
                ("clf", RandomForestClassifier())])

RFparameters={
    "clf__criterion":["gini","entropy"],
    "clf__min_samples_split":[6,8,9,10,12,20]
}
RF_clf = GridSearchCV(RF_clf,RFparameters, cv = 5, n_jobs=-1)
RF_clf=RF_clf.fit(review100, review_text_train_meta["rating"])
RF_clf.best_score_

H:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7399173435941285

In [47]:
RF_clf.best_params_

{'clf__criterion': 'entropy', 'clf__min_samples_split': 20}

# Final model

In [73]:
RF_clf = RandomForestClassifier(criterion="entropy", min_samples_split=9) # vect50

SVM_clf = SVM_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                ("tfidf",TfidfTransformer(use_idf=True)),
                ("clf", svm.SVC(kernel='linear', gamma=0.6, C=1.0))]) # countvect

NB_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                ("tfidf",TfidfTransformer(use_idf=False)),
                ("clf", MultinomialNB(alpha=0.1, fit_prior=False))]) # countvect

MLP_clf = Pipeline([('vect',CountVectorizer(stop_words="english",decode_error='ignore')),
                    ("tfidf",TfidfTransformer(use_idf=True)),
                    ('clf',MLPClassifier(activation='relu', alpha=1e-05, 
                                         hidden_layer_sizes=(80,100,100), solver='lbfgs'))]) # countvect

In [ ]:
NB=NB_clf.fit(review, meta["rating"])

# STACKING

In [13]:
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
#stacking
def stacking(review,vec50,meta,predictreview,predictmeta):
    #input: review - raw train text
    #input: meta - include rating
    #input: vec50 for Rndom forest
    #find train dataset's prediction

    RF_clf = RandomForestClassifier(criterion="entropy", min_samples_split=9) # vect50

    SVM_clf = SVM_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                    ("tfidf",TfidfTransformer(use_idf=True)),
                    ("clf", svm.SVC(kernel='linear', gamma=0.6, C=1.0))]) # countvect

    NB_clf=Pipeline([("vect", CountVectorizer(stop_words="english", decode_error="ignore")), 
                    ("tfidf",TfidfTransformer(use_idf=False)),
                    ("clf", MultinomialNB(alpha=0.1, fit_prior=False))]) # countvect

    MLP_clf = Pipeline([('vect',CountVectorizer(stop_words="english",decode_error='ignore')),
                        ("tfidf",TfidfTransformer(use_idf=True)),
                        ('clf',MLPClassifier(activation='relu', alpha=1e-05, 
                         hidden_layer_sizes=(80,100,100), solver='lbfgs'))]) # countvect
    print(review, meta["rating"])
    #get the predicted results of train
    NB=NB_clf.fit(review, meta["rating"])
    RF=RF_clf.fit(vec50,meta["rating"])
    SVM=SVM_clf.fit(review,meta["rating"])
    MLP=MLP_clf.fit(review,meta["rating"])
    

    otherinfo = meta.drop(['rating'], axis=1)
    
    other=svm.SVC(kernel='rbf', gamma=0.7, C=1.0)
    
    other.fit(otherinfo,df["rating"])

    NBresult=NB.predict(reivew, meta["rating"])
    SVMresult=SVM.predict(reivew, meta["rating"])
    RFresult=RF.predict(reivew, meta["rating"])
    MLPresult=MLP.predict(reivew, meta["rating"])
    finaldf = Dataframe()
    finaldf.join(NBresult)
    finaldf.join(SVMresult)
    finaldf.join(otherresult)
    finaldf.join(MLPresult)
    finaldf.join(RFresult)
    
    final=LogisticRegression()
    final.fit(finaldf,df["rating"])
    
    #predict
    predictdf=Dataframe()
    #get the predicted results of test
    predictdf.join(NB.predict(predictreview))
    predictdf.join(SVM.predict(predictreview))
    predictdf.join(RF.predict(predictreview))
    predictdf.join(MLP.predict(predictreview))

    predictdf.join(other.predict(predictmeta.drop(["rating"], axis=1)))
    result=final.predict(predictdf)

    return result


In [14]:

print(stacking(review_text_train, review50, review_text_train_meta, review_text_train,review_text_train_meta ))

                                                  review
0      dear longman & eagle.......you've left me no c...
1      Delish. The hubby and I wanted to do brunch on...
2      yep, I've giving Yolk 5 stars. It's just reall...
3      Meat, meat, meat. It's meat-tastic. So much me...
4      I caught up with the law school girls on a Sat...
5      Man I love Doug and his delicious sausages... ...
6      My boyfriend and I have started going to Nana'...
7      Overall, the food wasn't memorable. On the tas...
8      The Bad Apple is truly kick-ass. There are so ...
9      Rewind back to 2007: This place is my favorite...
10     It's really hard to say one restaurant in Chic...
11     If you asked me to review Icosium Kafe two wee...
12     This place was fantastic! Our waitress was ver...
13     I eat breakfast here at least once a week, alt...
14     I'm not a big Deep Dish Pizza guy, but this pl...
15     Finally, al la carte card #1 used! Had the ple...
16     I loved Chicago Q! Highl

ValueError: Found input variables with inconsistent numbers of samples: [1, 28068]